In [ ]:
def rand_primes(size):
    """Generates random primes with q < p < 2q. Taken from https://github.com/mseckept/generalized-wiener-attack"""
    p = random_prime(1 << (size - 1), 1 << size)
    while True:
        q = random_prime(1 << (size - 1), 1 << size)
        if p < q:
            p, q = q, p
            if q < p < 2*q:
                break
    return p,q

In [107]:
def generate_vulnerable_rsa_key(bits=256):
    """
    Generates a large, vulnerable RSA key pair.
    """
    print(f"--- 1. Generating a Vulnerable {bits}-bit RSA Key ---")

    # Generate large random primes, satifying q < p < 2q
    p, q = rand_primes(bits//2);
    n = p * q
    phi_n = (p - 1) * (q - 1)

    # Calculate the maximum d allowed by Wiener's condition: d < (1/3) * n^(1/4)
    max_d = Integer(round(1/3 * n**(1/4)))

    # Choose a small 'd' to ensure the attack succeeds
    d = random_prime(max_d // 100, lbound=1)

    # Calculate the public exponent 'e' using the optimized modular inverse
    e = inverse_mod(d, phi_n)

    print(f"Modulus n bit length: {n.nbits()}")
    print(f"Maximum safe d (for Wiener's attack): {max_d.nbits()} bits")
    print(f"Chosen d: {d} ({d.nbits()} bits)")

    return n, e, d, p, q

In [109]:
def wiener_attack(n, e):
    """
    Implements Wiener's attack by checking the convergents of e/n.
    """
    print("\n--- 2. Starting Wiener's Attack ---")

    # The target rational number for continued fraction expansion
    target_fraction = Rational((e, n))

    # Sage's continued_fraction() function returns the list of coefficients (a_i)
    cf_expansion = continued_fraction(target_fraction)
    print(f"Continued Fraction Expansion Coefficients (a_i): {cf_expansion}")
 
    # Iterate through the convergents (k/d_cand) of the continued fraction of e/n
    # Convergents are fractions that closely approximate the target_fraction.
    # The key d is guaranteed to be the denominator of one of these (Legendre Theorem).
    
    print("Testing Convergents...")
    
    # The .convergents() method returns the sequence of convergents as rational numbers
    convergents_list = list(cf_expansion.convergents())
    for i, convergent in enumerate(convergents_list):
        # The convergent is k_i / d_i
        k = convergent.numerator()
        d_cand = convergent.denominator()
        # Skip zero (First convergent)
        if k == 0:
            continue
        # Test 1: Check if (ed - 1) is divisible by k.
        # If true, phi_cand = (ed - 1) / k should be an integer.
        
        if (e * d_cand - 1) % k == 0:
            phi_cand = (e * d_cand - 1) // k
            # Test 2: Verify if this phi_cand can correctly factor n.
            # We know: p+q = n - phi_cand + 1
            # We solve the quadratic: x^2 - (p+q)x + n = 0
            
            sum_pq = n - phi_cand + 1
            
            # The discriminant of the quadratic formula: Delta = (p+q)^2 - 4n
            discriminant = sum_pq^2 - 4 * n
            
            # If Delta is a perfect square, p and q are integers.
            if discriminant >= 0 and isqrt(discriminant)^2 == discriminant:
                sqrt_discriminant = isqrt(discriminant)
                
                # Roots of the quadratic (the factors p and q)
                p_cand = (sum_pq + sqrt_discriminant) // 2
                q_cand = (sum_pq - sqrt_discriminant) // 2
                
                # Final check: are p and q factors of n and prime?
                if p_cand * q_cand == n and p_cand > 1 and q_cand > 1:
                    
                    # Found the secret key!
                    print(f"\n[SUCCESS] Key found at convergent #{i+1}")
                    print(f"Candidate d: {d_cand}")
                    print(f"Candidate p: {p_cand}")
                    print(f"Candidate q: {q_cand}")
                    return d_cand, p_cand, q_cand
        
    print("\n[FAILURE] Attack failed to find the key within the convergents.")
    return None, None, None

In [113]:
# 1. Generate the vulnerable key
n, e, d_true, p_true, q_true = generate_vulnerable_rsa_key(bits=512)

# 2. Run the attack
d_found, p_found, q_found = wiener_attack(n, e)

# 3. Verification
print("\n--- 3. Verification ---")
if d_found is not None:
    print(f"True d: {d_true}")
    print(f"Found d: {d_found}")
    print(f"Match: {d_true == d_found}")
    
    # Sanity check on factorization
    print(f"True factors: ({p_true}, {q_true})")
    print(f"Found factors: ({p_found}, {q_found})")
    print(f"Factors Match: {(p_true == p_found and q_true == q_found) or (p_true == q_found and q_true == p_found)}")
    
    # --- NEW CRYPTOGRAPHIC CHECK ---
    phi_found = (p_found - 1) * (q_found - 1)
    e_inverse = inverse_mod(e, phi_found)
    print(f"\n[CRYPTOGRAPHIC CHECK]")
    print(f"Inverse of e mod phi: {e_inverse}")
    print(f"d_found matches inverse: {d_found == e_inverse}")

else:
    print("Attack failed or a bug occurred in the implementation logic.")

--- 1. Generating a Vulnerable 512-bit RSA Key ---
Modulus n bit length: 509
Maximum safe d (for Wiener's attack): 126 bits
Chosen d: 216646407576704651342999062549046009 (118 bits)

--- 2. Starting Wiener's Attack ---
Continued Fraction Expansion Coefficients (a_i): [0; 13, 4, 1, 1, 19, 1, 2, 2, 2, 694, 2, 1, 2, 3, 43, 1, 2, 2, 1, 4, 1, 2, 1, 2, 1, 8, 5, 1, 1, 1, 1, 6, 1, 1, 7, 1, 3, 1, 2, 89, 1, 5, 28, 3, 1, 1, 1, 2, 1, 14, 1, 5, 12, 9, 7, 1, 10, 2, 15, 2, 1, 22, 7, 1, 1, 5436535, 6, 1, 1, 1, 1, 19, 1, 21, 1, 3, 1, 4, 2, 1, 3, 1, 1, 5, 1, 1, 3, 3, 12, 6, 23, 2, 12, 1, 1, 1, 22, 1, 5, 1, 5, 1, 6, 6, 1, 13, 56, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 2, 15, 5, 20, 1, 1, 3, 10, 3, 2, 1, 1, 3, 1, 11, 1, 3, 3, 1, 23, 1, 142, 2, 1, 10, 1, 1, 2, 4, 1, 5, 1, 2, 4, 1, 1, 3, 1, 1, 1, 12, 1, 2, 1, 32, 1, 3, 1, 1, 2, 1, 22, 3, 3, 17, 2, 1, 1, 2, 2, 1, 31, 1, 8, 7, 1, 2, 1, 2, 2, 2, 14, 2, 1, 4, 1, 9, 5, 1, 1, 6, 1, 6, 1, 45, 1, 18, 1, 1, 9, 2, 1, 3, 4, 1, 14, 1, 17, 2, 2, 3, 1, 2, 2, 6, 1, 1, 1, 5, 13, 20